In [1]:
import numpy as np
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('train_df.csv',encoding='euc_kr')
test = pd.read_csv('test_df.csv',encoding='euc_kr')
submission = pd.read_csv('sample_submission.csv')

### 파생변수 생성

In [10]:
train_dict = {
    'INVC_CONT':['mean','max','sum']
}

train_group = train.groupby('SEND_SPG_INNB')
train_agg = train_group.agg(train_dict)

train_agg.columns=['SEND_'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on='SEND_SPG_INNB', how='left')
test= test.merge(train_agg, on='SEND_SPG_INNB', how='left')

In [11]:
train_dict = {
    'INVC_CONT':['mean','max','sum']
}

train_group = train.groupby('DL_GD_LCLS_NM')
train_agg = train_group.agg(train_dict)

train_agg.columns=['LCLS_'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on='DL_GD_LCLS_NM', how='left')
test= test.merge(train_agg, on='DL_GD_LCLS_NM', how='left')

In [12]:
train_dict = {
    'INVC_CONT':['mean','max','sum']
}

train_group = train.groupby('DL_GD_MCLS_NM')
train_agg = train_group.agg(train_dict)

train_agg.columns=['MCLS_'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on='DL_GD_MCLS_NM', how='left')
test= test.merge(train_agg, on='DL_GD_MCLS_NM', how='left')

In [13]:
train_dict = {
    'INVC_CONT':['mean','max','sum']
}

train_group = train.groupby(['SEND_SPG_INNB', 'DL_GD_LCLS_NM'])
train_agg = train_group.agg(train_dict)

train_agg.columns=['SEND_LCLS_'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on=['SEND_SPG_INNB','DL_GD_LCLS_NM'], how='left')
test= test.merge(train_agg, on=['SEND_SPG_INNB','DL_GD_LCLS_NM'], how='left')

In [14]:
train_dict = {
    'INVC_CONT':['mean','max','sum']
}

train_group = train.groupby(['SEND_SPG_INNB', 'DL_GD_MCLS_NM'])
train_agg = train_group.agg(train_dict)

train_agg.columns=['SEND_MCLS_'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on=['SEND_SPG_INNB','DL_GD_MCLS_NM'], how='left')
test= test.merge(train_agg, on=['SEND_SPG_INNB','DL_GD_MCLS_NM'], how='left')

### 결측치 확인

In [15]:
train.isnull().sum()

index                       0
SEND_SPG_INNB               0
REC_SPG_INNB                0
DL_GD_LCLS_NM               0
DL_GD_MCLS_NM               0
INVC_CONT                   0
SEND_INVC_CONT_MEAN         0
SEND_INVC_CONT_MAX          0
SEND_INVC_CONT_SUM          0
LCLS_INVC_CONT_MEAN         0
LCLS_INVC_CONT_MAX          0
LCLS_INVC_CONT_SUM          0
MCLS_INVC_CONT_MEAN         0
MCLS_INVC_CONT_MAX          0
MCLS_INVC_CONT_SUM          0
SEND_LCLS_INVC_CONT_MEAN    0
SEND_LCLS_INVC_CONT_MAX     0
SEND_LCLS_INVC_CONT_SUM     0
SEND_MCLS_INVC_CONT_MEAN    0
SEND_MCLS_INVC_CONT_MAX     0
SEND_MCLS_INVC_CONT_SUM     0
dtype: int64

In [16]:
test.isnull().sum()

index                         0
SEND_SPG_INNB                 0
REC_SPG_INNB                  0
DL_GD_LCLS_NM                 0
DL_GD_MCLS_NM                 0
SEND_INVC_CONT_MEAN         313
SEND_INVC_CONT_MAX          313
SEND_INVC_CONT_SUM          313
LCLS_INVC_CONT_MEAN           0
LCLS_INVC_CONT_MAX            0
LCLS_INVC_CONT_SUM            0
MCLS_INVC_CONT_MEAN           0
MCLS_INVC_CONT_MAX            0
MCLS_INVC_CONT_SUM            0
SEND_LCLS_INVC_CONT_MEAN    339
SEND_LCLS_INVC_CONT_MAX     339
SEND_LCLS_INVC_CONT_SUM     339
SEND_MCLS_INVC_CONT_MEAN    405
SEND_MCLS_INVC_CONT_MAX     405
SEND_MCLS_INVC_CONT_SUM     405
dtype: int64

### 결측치 처리

In [17]:
test['SEND_INVC_CONT_MEAN'] = test['SEND_INVC_CONT_MEAN'].fillna(train['SEND_INVC_CONT_MEAN'].mean())
test['SEND_INVC_CONT_MAX'] = test['SEND_INVC_CONT_MAX'].fillna(train['SEND_INVC_CONT_MAX'].mean())
test['SEND_INVC_CONT_SUM'] = test['SEND_INVC_CONT_SUM'].fillna(train['SEND_INVC_CONT_SUM'].mean())

In [18]:
test['SEND_LCLS_INVC_CONT_MEAN'] = test['SEND_LCLS_INVC_CONT_MEAN'].fillna(train['SEND_LCLS_INVC_CONT_MEAN'].mean())
test['SEND_LCLS_INVC_CONT_MAX'] = test['SEND_LCLS_INVC_CONT_MAX'].fillna(train['SEND_LCLS_INVC_CONT_MAX'].mean())
test['SEND_LCLS_INVC_CONT_SUM'] = test['SEND_LCLS_INVC_CONT_SUM'].fillna(train['SEND_LCLS_INVC_CONT_SUM'].mean())

In [19]:
test['SEND_MCLS_INVC_CONT_MEAN'] = test['SEND_MCLS_INVC_CONT_MEAN'].fillna(train['SEND_MCLS_INVC_CONT_MEAN'].mean())
test['SEND_MCLS_INVC_CONT_MAX'] = test['SEND_MCLS_INVC_CONT_MAX'].fillna(train['SEND_MCLS_INVC_CONT_MAX'].mean())
test['SEND_MCLS_INVC_CONT_SUM'] = test['SEND_MCLS_INVC_CONT_SUM'].fillna(train['SEND_MCLS_INVC_CONT_SUM'].mean())

In [20]:
data= pd.concat([train,test])

In [21]:
# 송하인-수하인 변수
data['SEND_REC']= data['SEND_SPG_INNB'].astype('str')+'-'+data['REC_SPG_INNB'].astype('str')

In [22]:
data.columns

Index(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB', 'DL_GD_LCLS_NM',
       'DL_GD_MCLS_NM', 'INVC_CONT', 'SEND_INVC_CONT_MEAN',
       'SEND_INVC_CONT_MAX', 'SEND_INVC_CONT_SUM', 'LCLS_INVC_CONT_MEAN',
       'LCLS_INVC_CONT_MAX', 'LCLS_INVC_CONT_SUM', 'MCLS_INVC_CONT_MEAN',
       'MCLS_INVC_CONT_MAX', 'MCLS_INVC_CONT_SUM', 'SEND_LCLS_INVC_CONT_MEAN',
       'SEND_LCLS_INVC_CONT_MAX', 'SEND_LCLS_INVC_CONT_SUM',
       'SEND_MCLS_INVC_CONT_MEAN', 'SEND_MCLS_INVC_CONT_MAX',
       'SEND_MCLS_INVC_CONT_SUM', 'SEND_REC'],
      dtype='object')

### Label Encoding

In [23]:
object_columns=['SEND_SPG_INNB','REC_SPG_INNB','DL_GD_LCLS_NM','DL_GD_MCLS_NM','SEND_REC']
for column in object_columns:
    data[column] = pd.factorize(data[column])[0]

In [24]:
data=data.drop('index',axis=1)

In [25]:
train = data[~data['INVC_CONT'].isnull()]
test = data[data['INVC_CONT'].isnull()]

In [26]:
train_label=train['INVC_CONT']
train = train.drop('INVC_CONT', axis=1)
test = test.drop('INVC_CONT', axis=1)

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32000 entries, 0 to 31999
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SEND_SPG_INNB             32000 non-null  int64  
 1   REC_SPG_INNB              32000 non-null  int64  
 2   DL_GD_LCLS_NM             32000 non-null  int64  
 3   DL_GD_MCLS_NM             32000 non-null  int64  
 4   SEND_INVC_CONT_MEAN       32000 non-null  float64
 5   SEND_INVC_CONT_MAX        32000 non-null  float64
 6   SEND_INVC_CONT_SUM        32000 non-null  float64
 7   LCLS_INVC_CONT_MEAN       32000 non-null  float64
 8   LCLS_INVC_CONT_MAX        32000 non-null  int64  
 9   LCLS_INVC_CONT_SUM        32000 non-null  int64  
 10  MCLS_INVC_CONT_MEAN       32000 non-null  float64
 11  MCLS_INVC_CONT_MAX        32000 non-null  int64  
 12  MCLS_INVC_CONT_SUM        32000 non-null  int64  
 13  SEND_LCLS_INVC_CONT_MEAN  32000 non-null  float64
 14  SEND_L

In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4640 entries, 0 to 4639
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SEND_SPG_INNB             4640 non-null   int64  
 1   REC_SPG_INNB              4640 non-null   int64  
 2   DL_GD_LCLS_NM             4640 non-null   int64  
 3   DL_GD_MCLS_NM             4640 non-null   int64  
 4   SEND_INVC_CONT_MEAN       4640 non-null   float64
 5   SEND_INVC_CONT_MAX        4640 non-null   float64
 6   SEND_INVC_CONT_SUM        4640 non-null   float64
 7   LCLS_INVC_CONT_MEAN       4640 non-null   float64
 8   LCLS_INVC_CONT_MAX        4640 non-null   int64  
 9   LCLS_INVC_CONT_SUM        4640 non-null   int64  
 10  MCLS_INVC_CONT_MEAN       4640 non-null   float64
 11  MCLS_INVC_CONT_MAX        4640 non-null   int64  
 12  MCLS_INVC_CONT_SUM        4640 non-null   int64  
 13  SEND_LCLS_INVC_CONT_MEAN  4640 non-null   float64
 14  SEND_LCL

## Modeling

### CatBoost

In [29]:
cat_features=['SEND_SPG_INNB','REC_SPG_INNB','DL_GD_LCLS_NM','DL_GD_MCLS_NM',
                'SEND_REC']

In [32]:
model = CatBoostRegressor(n_estimators=2000, learning_rate=0.02, cat_features=cat_features)
model.fit(train, train_label)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 5.7166112	total: 138ms	remaining: 4m 36s
1:	learn: 5.6815308	total: 259ms	remaining: 4m 19s
2:	learn: 5.6497288	total: 358ms	remaining: 3m 58s
3:	learn: 5.6175879	total: 482ms	remaining: 4m
4:	learn: 5.5858639	total: 615ms	remaining: 4m 5s
5:	learn: 5.5548531	total: 728ms	remaining: 4m 1s
6:	learn: 5.5249746	total: 825ms	remaining: 3m 54s
7:	learn: 5.4952594	total: 923ms	remaining: 3m 49s
8:	learn: 5.4682578	total: 1.02s	remaining: 3m 44s
9:	learn: 5.4409716	total: 1.11s	remaining: 3m 40s
10:	learn: 5.4139500	total: 1.2s	remaining: 3m 36s
11:	learn: 5.3897677	total: 1.31s	remaining: 3m 36s
12:	learn: 5.3645296	total: 1.41s	remaining: 3m 35s
13:	learn: 5.3401996	total: 1.52s	remaining: 3m 36s
14:	learn: 5.3155972	total: 1.64s	remaining: 3m 36s
15:	learn: 5.2934653	total: 1.75s	remaining: 3m 36s
16:	learn: 5.2724560	total: 1.83s	remaining: 3m 33s
17:	learn: 5.2505354	total: 1.93s	remaining: 3m 32s
18:	learn: 5.2289229	total: 2.03s	remaining: 3m 32s
19:	learn: 5.2079812	total: 2

160:	learn: 4.4118302	total: 18.8s	remaining: 3m 34s
161:	learn: 4.4106880	total: 18.9s	remaining: 3m 34s
162:	learn: 4.4085252	total: 19s	remaining: 3m 34s
163:	learn: 4.4068421	total: 19.1s	remaining: 3m 34s
164:	learn: 4.4048824	total: 19.2s	remaining: 3m 34s
165:	learn: 4.4018463	total: 19.3s	remaining: 3m 33s
166:	learn: 4.4006002	total: 19.5s	remaining: 3m 33s
167:	learn: 4.3977831	total: 19.6s	remaining: 3m 33s
168:	learn: 4.3970201	total: 19.7s	remaining: 3m 33s
169:	learn: 4.3961139	total: 19.8s	remaining: 3m 33s
170:	learn: 4.3952603	total: 19.9s	remaining: 3m 33s
171:	learn: 4.3942565	total: 20.1s	remaining: 3m 33s
172:	learn: 4.3924882	total: 20.1s	remaining: 3m 32s
173:	learn: 4.3910823	total: 20.2s	remaining: 3m 32s
174:	learn: 4.3901144	total: 20.3s	remaining: 3m 31s
175:	learn: 4.3895690	total: 20.4s	remaining: 3m 31s
176:	learn: 4.3886044	total: 20.5s	remaining: 3m 31s
177:	learn: 4.3876167	total: 20.7s	remaining: 3m 31s
178:	learn: 4.3863974	total: 20.8s	remaining: 3m

318:	learn: 4.2378592	total: 33.6s	remaining: 2m 56s
319:	learn: 4.2376145	total: 33.6s	remaining: 2m 56s
320:	learn: 4.2371300	total: 33.7s	remaining: 2m 56s
321:	learn: 4.2368145	total: 33.8s	remaining: 2m 56s
322:	learn: 4.2365252	total: 33.9s	remaining: 2m 55s
323:	learn: 4.2356346	total: 34s	remaining: 2m 55s
324:	learn: 4.2354461	total: 34s	remaining: 2m 55s
325:	learn: 4.2349653	total: 34.1s	remaining: 2m 55s
326:	learn: 4.2346702	total: 34.2s	remaining: 2m 54s
327:	learn: 4.2344946	total: 34.3s	remaining: 2m 54s
328:	learn: 4.2341036	total: 34.4s	remaining: 2m 54s
329:	learn: 4.2337171	total: 34.5s	remaining: 2m 54s
330:	learn: 4.2329025	total: 34.5s	remaining: 2m 54s
331:	learn: 4.2327115	total: 34.6s	remaining: 2m 53s
332:	learn: 4.2323100	total: 34.7s	remaining: 2m 53s
333:	learn: 4.2319902	total: 34.7s	remaining: 2m 53s
334:	learn: 4.2309675	total: 34.8s	remaining: 2m 53s
335:	learn: 4.2303641	total: 34.9s	remaining: 2m 52s
336:	learn: 4.2299246	total: 35s	remaining: 2m 52s

476:	learn: 4.1488045	total: 47.8s	remaining: 2m 32s
477:	learn: 4.1485939	total: 47.8s	remaining: 2m 32s
478:	learn: 4.1478167	total: 47.9s	remaining: 2m 32s
479:	learn: 4.1475996	total: 48s	remaining: 2m 31s
480:	learn: 4.1471333	total: 48.1s	remaining: 2m 31s
481:	learn: 4.1468040	total: 48.2s	remaining: 2m 31s
482:	learn: 4.1463515	total: 48.2s	remaining: 2m 31s
483:	learn: 4.1454658	total: 48.3s	remaining: 2m 31s
484:	learn: 4.1444040	total: 48.4s	remaining: 2m 31s
485:	learn: 4.1436936	total: 48.5s	remaining: 2m 30s
486:	learn: 4.1435313	total: 48.5s	remaining: 2m 30s
487:	learn: 4.1432687	total: 48.6s	remaining: 2m 30s
488:	learn: 4.1427955	total: 48.7s	remaining: 2m 30s
489:	learn: 4.1424963	total: 48.8s	remaining: 2m 30s
490:	learn: 4.1418858	total: 48.9s	remaining: 2m 30s
491:	learn: 4.1417840	total: 48.9s	remaining: 2m 29s
492:	learn: 4.1409252	total: 49s	remaining: 2m 29s
493:	learn: 4.1406252	total: 49.1s	remaining: 2m 29s
494:	learn: 4.1403273	total: 49.2s	remaining: 2m 2

...

1243:	learn: 3.9774670	total: 1m 55s	remaining: 1m 10s
1244:	learn: 3.9771908	total: 1m 55s	remaining: 1m 9s
1245:	learn: 3.9771485	total: 1m 55s	remaining: 1m 9s
1246:	learn: 3.9770402	total: 1m 55s	remaining: 1m 9s
1247:	learn: 3.9769629	total: 1m 55s	remaining: 1m 9s
1248:	learn: 3.9769212	total: 1m 55s	remaining: 1m 9s
1249:	learn: 3.9768804	total: 1m 55s	remaining: 1m 9s
1250:	learn: 3.9767665	total: 1m 55s	remaining: 1m 9s
1251:	learn: 3.9766061	total: 1m 56s	remaining: 1m 9s
1252:	learn: 3.9759252	total: 1m 56s	remaining: 1m 9s
1253:	learn: 3.9758850	total: 1m 56s	remaining: 1m 9s
1254:	learn: 3.9758143	total: 1m 56s	remaining: 1m 9s
1255:	learn: 3.9757747	total: 1m 56s	remaining: 1m 9s
1256:	learn: 3.9757357	total: 1m 56s	remaining: 1m 8s
1257:	learn: 3.9755996	total: 1m 56s	remaining: 1m 8s
1258:	learn: 3.9755307	total: 1m 56s	remaining: 1m 8s
1259:	learn: 3.9754921	total: 1m 56s	remaining: 1m 8s
1260:	learn: 3.9754221	total: 1m 57s	remaining: 1m 8s
1261:	learn: 3.9753125	tota

1401:	learn: 3.9561366	total: 2m 10s	remaining: 55.6s
1402:	learn: 3.9558805	total: 2m 10s	remaining: 55.5s
1403:	learn: 3.9558672	total: 2m 10s	remaining: 55.4s
1404:	learn: 3.9557785	total: 2m 10s	remaining: 55.3s
1405:	learn: 3.9557643	total: 2m 10s	remaining: 55.2s
1406:	learn: 3.9557367	total: 2m 10s	remaining: 55.1s
1407:	learn: 3.9555887	total: 2m 10s	remaining: 55s
1408:	learn: 3.9555461	total: 2m 10s	remaining: 54.9s
1409:	learn: 3.9554347	total: 2m 11s	remaining: 54.8s
1410:	learn: 3.9553663	total: 2m 11s	remaining: 54.8s
1411:	learn: 3.9552758	total: 2m 11s	remaining: 54.7s
1412:	learn: 3.9552298	total: 2m 11s	remaining: 54.6s
1413:	learn: 3.9549673	total: 2m 11s	remaining: 54.5s
1414:	learn: 3.9546022	total: 2m 11s	remaining: 54.4s
1415:	learn: 3.9543854	total: 2m 11s	remaining: 54.3s
1416:	learn: 3.9543344	total: 2m 11s	remaining: 54.2s
1417:	learn: 3.9541539	total: 2m 11s	remaining: 54.1s
1418:	learn: 3.9541266	total: 2m 11s	remaining: 54s
1419:	learn: 3.9538942	total: 2m

1555:	learn: 3.9377671	total: 2m 33s	remaining: 43.7s
1556:	learn: 3.9376987	total: 2m 33s	remaining: 43.6s
1557:	learn: 3.9376157	total: 2m 33s	remaining: 43.5s
1558:	learn: 3.9374519	total: 2m 33s	remaining: 43.4s
1559:	learn: 3.9372558	total: 2m 33s	remaining: 43.3s
1560:	learn: 3.9370954	total: 2m 33s	remaining: 43.2s
1561:	learn: 3.9370470	total: 2m 33s	remaining: 43.1s
1562:	learn: 3.9370140	total: 2m 33s	remaining: 43.1s
1563:	learn: 3.9367046	total: 2m 34s	remaining: 43s
1564:	learn: 3.9366720	total: 2m 34s	remaining: 42.9s
1565:	learn: 3.9366330	total: 2m 34s	remaining: 42.8s
1566:	learn: 3.9365500	total: 2m 34s	remaining: 42.7s
1567:	learn: 3.9365236	total: 2m 34s	remaining: 42.6s
1568:	learn: 3.9364812	total: 2m 34s	remaining: 42.5s
1569:	learn: 3.9364600	total: 2m 35s	remaining: 42.5s
1570:	learn: 3.9362689	total: 2m 35s	remaining: 42.4s
1571:	learn: 3.9362412	total: 2m 35s	remaining: 42.3s
1572:	learn: 3.9362206	total: 2m 35s	remaining: 42.2s
1573:	learn: 3.9359007	total: 

1708:	learn: 3.9208623	total: 2m 50s	remaining: 29.1s
1709:	learn: 3.9207117	total: 2m 50s	remaining: 29s
1710:	learn: 3.9206886	total: 2m 51s	remaining: 28.9s
1711:	learn: 3.9206493	total: 2m 51s	remaining: 28.8s
1712:	learn: 3.9204805	total: 2m 51s	remaining: 28.7s
1713:	learn: 3.9200883	total: 2m 51s	remaining: 28.6s
1714:	learn: 3.9200099	total: 2m 51s	remaining: 28.5s
1715:	learn: 3.9199724	total: 2m 51s	remaining: 28.4s
1716:	learn: 3.9199187	total: 2m 51s	remaining: 28.3s
1717:	learn: 3.9198467	total: 2m 51s	remaining: 28.2s
1718:	learn: 3.9196359	total: 2m 51s	remaining: 28.1s
1719:	learn: 3.9195674	total: 2m 51s	remaining: 28s
1720:	learn: 3.9194600	total: 2m 52s	remaining: 27.9s
1721:	learn: 3.9193866	total: 2m 52s	remaining: 27.8s
1722:	learn: 3.9192231	total: 2m 52s	remaining: 27.7s
1723:	learn: 3.9189488	total: 2m 52s	remaining: 27.6s
1724:	learn: 3.9187822	total: 2m 52s	remaining: 27.5s
1725:	learn: 3.9187150	total: 2m 52s	remaining: 27.4s
1726:	learn: 3.9186489	total: 2m

1863:	learn: 3.9013871	total: 3m 9s	remaining: 13.9s
1864:	learn: 3.9012463	total: 3m 9s	remaining: 13.8s
1865:	learn: 3.9011383	total: 3m 10s	remaining: 13.7s
1866:	learn: 3.9010317	total: 3m 10s	remaining: 13.5s
1867:	learn: 3.9009746	total: 3m 10s	remaining: 13.4s
1868:	learn: 3.9008951	total: 3m 10s	remaining: 13.3s
1869:	learn: 3.9008216	total: 3m 10s	remaining: 13.2s
1870:	learn: 3.9006925	total: 3m 10s	remaining: 13.1s
1871:	learn: 3.9006373	total: 3m 10s	remaining: 13s
1872:	learn: 3.9005600	total: 3m 10s	remaining: 12.9s
1873:	learn: 3.9004750	total: 3m 10s	remaining: 12.8s
1874:	learn: 3.9004125	total: 3m 10s	remaining: 12.7s
1875:	learn: 3.9003369	total: 3m 10s	remaining: 12.6s
1876:	learn: 3.9002973	total: 3m 10s	remaining: 12.5s
1877:	learn: 3.9001019	total: 3m 11s	remaining: 12.4s
1878:	learn: 3.8999474	total: 3m 11s	remaining: 12.3s
1879:	learn: 3.8997521	total: 3m 11s	remaining: 12.2s
1880:	learn: 3.8995683	total: 3m 11s	remaining: 12.1s
1881:	learn: 3.8995106	total: 3m

In [33]:
predict = model.predict(test)
predict

array([5.16110388, 4.39172574, 5.19648643, ..., 4.59029243, 4.6882352 ,
       3.90611269])

In [34]:
submission['INVC_CONT'] = predict

In [35]:
submission.to_csv('madrid.csv',index = False)